<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/vector_drug_CLIPModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/openai/clip-vit-large-patch14

In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

In [ ]:
print("Image-text similarity scores:", logits_per_image)
print("Label probabilities:", probs)

Image-text similarity scores: tensor([[18.9041, 11.7159]], grad_fn=<TBackward0>)
Label probabilities: tensor([[9.9925e-01, 7.5487e-04]], grad_fn=<SoftmaxBackward0>)


# ลอง

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/drug/data 50 class add_augment.zip'

In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from datetime import datetime

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Path to the main directory containing 50 subfolders
main_directory = "/content/data 50 class add_augment - Copy"

# Prepare lists to store rows for CSV
rows = []

# Start timer for the whole process
start_time = datetime.now()

# Collect images and text labels from the subfolders and convert them to vectors
for folder_name in os.listdir(main_directory):
    folder_path = os.path.join(main_directory, folder_name)
    if os.path.isdir(folder_path):
        text_label = folder_name

        # Start timer for the current folder
        folder_start_time = datetime.now()

        # Collect all images from the current folder
        image_files = [os.path.join(folder_path, image_name) for image_name in os.listdir(folder_path)]

        for image_path in image_files:
            # Open the image
            image = Image.open(image_path)

            # Prepare the inputs for the CLIP model
            inputs = processor(text=[text_label], images=image, return_tensors="pt", padding=True)

            # Forward pass through the model
            with torch.no_grad():
                outputs = model(**inputs)

            # Get the image and text embeddings
            image_embedding = outputs.image_embeds.squeeze().tolist()
            text_embedding = outputs.text_embeds.squeeze().tolist()

            # Create a row for the CSV
            row = [text_label, image_path] + image_embedding + text_embedding
            rows.append(row)

        # End timer for the current folder
        folder_end_time = datetime.now()
        folder_duration = folder_end_time - folder_start_time
        print(f"Converted vectors for folder: {folder_name} in {folder_duration}")

# End timer for the whole process
end_time = datetime.now()
total_duration = end_time - start_time

# Define the header for the CSV file
header = ['TextLabel', 'ImagePath'] + [f'ImageEmbed_{i}' for i in range(len(image_embedding))] + [f'TextEmbed_{i}' for i in range(len(text_embedding))]

# Write the data to a CSV file
csv_file_path = '/content/class_vectors.csv'
df = pd.DataFrame(rows, columns=header)
df.to_csv(csv_file_path, index=False)

print(f"Vectors have been successfully saved to: {csv_file_path}")
print(f"Total time for processing all folders: {total_duration}")

Converted vectors for folder: Janumet50_1000mg in 0:02:29.272910
Converted vectors for folder: Prenolol50mg in 0:02:22.600846
Converted vectors for folder: Glyxambi25_5mg in 0:02:30.322592
Converted vectors for folder: Madiplot20mg in 0:02:26.075998
Converted vectors for folder: GlucophageXR1000mg in 0:02:25.581881
Converted vectors for folder: Amlopine10mg in 0:02:23.748385
Converted vectors for folder: Caduet5_10mg in 0:02:23.690538
Converted vectors for folder: Glucophage500mg in 0:02:25.931969
Converted vectors for folder: Betalol10mg in 0:02:23.035979


In [ ]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Path to your local image file
image_path = "/content/data 50 class add_augment - Copy/Betalol40mg/Betalol40mg_bright.jpg"

# Open the image
image = Image.open(image_path)

# Define multiple text labels for comparison
text_labels = ['Amlopine_10mg', 'Amlopine_5mg', 'Anapril_5mg', 'Betalol-10-mg', 'Betalol-40-mg',
               'Blopress-16-mg', 'Blopress-8-mg', 'Blopress-Plus-16-mg', 'Blopress-Plus-8-mg',
               'Caduet-10_10-mg', 'Caduet-5_10-mg', 'Daonil-5-mg', 'Diamicron-MR-60-mg',
               'Diovan-160-mg', 'Diovan-80-mg', 'Forxiga-10-mg', 'Galvus-50-mg', 'Galvus_Met_50_1000mg',
               'Gliclazide_80mg', 'Gliparil-2-mg', 'Glucophage-500-mg', 'Glucophage-850-mg',
               'Glucophage-XR-1000-mg', 'Glucophage-XR-750-mg', 'Glyxambi-25_5-mg', 'Janumet-50_1000-mg',
               'Januvia-100-mg', 'Jardiance-10-mg', 'Jardiance-25-mg', 'Jardiance-Duo-12_5_1000-mg',
               'Lanzaar-100-mg', 'Lercadip-20-mg', 'Madiplot-10-mg', 'Madiplot_20mg',
               'Metoprolol_Stada_100mg', 'Micardis_40mg', 'Micardis_Plus_80_12_5mg', 'Minidiab_5mg',
               'Novonorm_1mg', 'Novonorm_2mg', 'Oseni_25_15mg', 'Poli_uretic', 'Prenolol_100mg',
               'Prenolol_25mg', 'Prenolol_50mg', 'Tanzaril_50mg', 'Utmos_15mg', 'Utmos_30mg',
               'Xigduo_XR_10_1000mg', 'Zanidip_10mg']

# Prepare the inputs for the CLIP model
inputs = processor(text=text_labels, images=image, return_tensors="pt", padding=True)

# Forward pass through the model
outputs = model(**inputs)

# Get the image-text similarity scores
logits_per_image = outputs.logits_per_image

# Compute the probabilities
probs = logits_per_image.softmax(dim=1)

# Get the top 5 similarity scores and their corresponding text labels
top_probs, top_indices = probs.topk(5, dim=1)

# Convert to lists for easier manipulation
top_probs = top_probs.squeeze().tolist()
top_indices = top_indices.squeeze().tolist()

# Print the top 5 similarity scores and their corresponding text labels in descending order
print("Top 5 similarity scores and corresponding text labels:")
for i, idx in enumerate(top_indices):
    print(f"{text_labels[idx]}: {top_probs[i]:.4f}")

Top 5 similarity scores and corresponding text labels:
Betalol-40-mg: 0.7764
Betalol-10-mg: 0.1812
Prenolol_50mg: 0.0250
Prenolol_100mg: 0.0084
Prenolol_25mg: 0.0051
